## Data Analysis On Electric Vehicle 

- It involves using different graphs and plots to help visualise the data and also uses statistical methods to draw inferences from the data.

- The goal of this analysis is not to arrive at a certain right answer or to confirm a pre-defined hypothesis. It is an exploratory process to draw inferences and get ideas on how the data can be further utilised to predict certain outcomes/develop ML models

## Introduction to the Electric Vehicles

Which is the best electric car? What would be the best option for the x person? 

- Which car?
** Mileage/Range 
** Pricing 
** How many seats
- How far is the charging station?
** Charging for the price?
** If charging station is supercharge - DURATION?
- Does the car manufature give goverment subsidy?

## About Dataset

We will be downloading the electric vehicle manufature data set from: 
https://documenter.getpostman.com/view/3967924/RW1hhvSR#5977afdf-8740-401a-9fd5-e926c983aa52
- Get Dealers

https://rapidapi.com/letscrape-6bRBa3QguO5/api/ev-charge-finder
- Charging Location 

### PROJECT OUTINE

The steps involved in the process of Exploratory Data Analysis are.,

- Import the required libraries and its dependencies.

- Download the dataset.

- Data preparation and cleaning it.

- Exploratory Analysis.

- Ask and solve questions from the data.

- Pictorial representation of data using visualization techniques.

## Steps in Exploratory Data Analysis
* Select the real-world data set from Kaggle/any other source.
* Download the dataset and convert it into a Pandas DataFrame.
* Perform data cleaning/deal with missing values using Pandas and NumPy.
* Plot interactive graphs to understand the data using visual libraries like Plotly.
* Ask and Answer interesting questions from the dataset.
* Summarize your key inferences and state the conclusion.
* Elaborate on ideas for future work and quote references.

## Installing Packages and Importing Libraries

In [1]:
#!pip install opendatasets jovian --quiet --upgrade

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


In [3]:
# Import the Rapid API key
from config import RapidAPIKey

# Import the PostMate/MarketCheck API key

## Step 1: Download the dataset

In [4]:
csv_file_path = '../Project_1/Electric_Vehicle_Population_Data (1).csv'
df = pd.read_csv(csv_file_path)
df

#/Users/eshapatel/Desktop/Project_1/Electric_Vehicle_Population_Data (1).csv

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,2C4RC1N71H,Kitsap,Bremerton,WA,98311.0,2017,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,33,0,23.0,349437882,POINT (-122.6466274 47.6341188),PUGET SOUND ENERGY INC,5.303509e+10
1,2C4RC1N7XL,Stevens,Colville,WA,99114.0,2020,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,32,0,7.0,154690532,POINT (-117.90431 48.547075),AVISTA CORP,5.306595e+10
2,KNDC3DLCXN,Yakima,Yakima,WA,98908.0,2022,KIA,EV6,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,14.0,219969144,POINT (-120.6027202 46.5965625),PACIFICORP,5.307700e+10
3,5YJ3E1EA0J,Kitsap,Bainbridge Island,WA,98110.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,23.0,476786887,POINT (-122.5235781 47.6293323),PUGET SOUND ENERGY INC,5.303509e+10
4,1N4AZ1CP7J,Thurston,Tumwater,WA,98501.0,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151,0,35.0,201185253,POINT (-122.89692 47.043535),PUGET SOUND ENERGY INC,5.306701e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159462,KM8JBDA2XP,Skamania,Underwood,WA,98651.0,2023,HYUNDAI,TUCSON,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,33,0,14.0,235949514,POINT (-121.5312858 45.7348285),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF S...,5.305995e+10
159463,1G1FZ6S02M,Skagit,Bow,WA,98232.0,2021,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,40.0,148544168,POINT (-122.440636 48.5613885),PUGET SOUND ENERGY INC,5.305795e+10
159464,YV4H60CX2P,King,Sammamish,WA,98029.0,2023,VOLVO,XC90,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,32,0,5.0,240200754,POINT (-121.9993659 47.5484866),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
159465,5YJ3E1EA7K,Whatcom,Bellingham,WA,98225.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220,0,40.0,156680590,POINT (-122.486115 48.761615),PUGET SOUND ENERGY INC||PUD NO 1 OF WHATCOM CO...,5.307300e+10


In [5]:
selected_columns = ['Model Year', 'Make', 'Model', 'Electric Vehicle Type', 'Electric Range', 'Base MSRP']
compressed_df = df[selected_columns]
compressed_df


,Model Year,Make,Model,Electric Vehicle Type,Electric Range,Base MSRP
0,2017,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),33,0
1,2020,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),32,0
2,2022,KIA,EV6,Battery Electric Vehicle (BEV),0,0
3,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),215,0
4,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),151,0
...,...,...,...,...,...,...
159462,2023,HYUNDAI,TUCSON,Plug-in Hybrid Electric Vehicle (PHEV),33,0
159463,2021,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),0,0
159464,2023,VOLVO,XC90,Plug-in Hybrid Electric Vehicle (PHEV),32,0
159465,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),220,0


In [6]:
filtered_df = compressed_df[compressed_df['Electric Vehicle Type'] == 'Battery Electric Vehicle (BEV)']
filtered_df

,Model Year,Make,Model,Electric Vehicle Type,Electric Range,Base MSRP
2,2022,KIA,EV6,Battery Electric Vehicle (BEV),0,0
3,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),215,0
4,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),151,0
5,2019,KIA,NIRO,Battery Electric Vehicle (BEV),239,0
7,2022,TESLA,MODEL Y,Battery Electric Vehicle (BEV),0,0
...,...,...,...,...,...,...
159458,2021,VOLKSWAGEN,ID.4,Battery Electric Vehicle (BEV),0,0
159461,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),0,0
159463,2021,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),0,0
159465,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),220,0


In [7]:
csv_file_path = '../Project_1/Electric Vehicles EV Taxes by State Details  Analysis.csv'
tax_rebate_per_state_df = pd.read_csv(csv_file_path)
tax_rebate_per_state_df

,State,EV Purchase Tax Credit,Additional EV Annual Registration Fee,Additional Hybrid Annual Registration Fee
0,Maine,"Up to 7,500 (e)",0,0
1,Oklahoma,"Up to 5,500",110,82
2,Oregon,"Up to 5,000 (b) (e)",76-91,0
3,New Jersey,"Up to 4,000 (b) (h)",0,0
4,Vermont,"Up to 4,000",0,0
5,Massachusetts,"Up to 3,500",0,0
6,Pennsylvania,"Up to 3,000",0,0
7,Rhode Island,"Up to 2,500",0,0
8,Kansas,"up to 2,400",60-70,20-30
9,New York,"Up to 2,000",0,0


In [8]:
tax_rebate_per_state_df.columns

Index(['State', 'EV Purchase Tax Credit',
       'Additional EV Annual Registration Fee',
       'Additional Hybrid Annual Registration Fee'],
      dtype='object')

In [9]:
selected_columns_for_tax_rebate= ['State', 'EV Purchase Tax Credit']
tax_rebate_compressed_df = tax_rebate_per_state_df[selected_columns_for_tax_rebate]
tax_rebate_compressed_df

,State,EV Purchase Tax Credit
0,Maine,"Up to 7,500 (e)"
1,Oklahoma,"Up to 5,500"
2,Oregon,"Up to 5,000 (b) (e)"
3,New Jersey,"Up to 4,000 (b) (h)"
4,Vermont,"Up to 4,000"
5,Massachusetts,"Up to 3,500"
6,Pennsylvania,"Up to 3,000"
7,Rhode Island,"Up to 2,500"
8,Kansas,"up to 2,400"
9,New York,"Up to 2,000"


In [10]:
csv_file_path = '../Project_1/EV_personal_use_tax _incentives.csv'
tax_rebate_per_model_df = pd.read_csv(csv_file_path)
tax_rebate_per_model_df



,manufacturer,model,model_year_introduced,curent_model-year,vehicle_type,tax_credit_amount,msrp_limit,assembled_in _usa
0,Audi,Q5 TFSI e Quattro (PHEV),NaN,2023,PHEV,7500,80000,True
1,BMW,330e,2021.0,2023,PHEV,5836,55000,True
2,BMW,X5 xDrive45e,2021.0,2023,PHEV,7500,80000,True
3,BMW,X5 xDrive50e,NaN,2024,PHEV,7500,80000,True
4,Cadillac,Lyriq,2022.0,2024,EV,7500,80000,True
5,Chevrolet,Bolt,2022.0,2024,EV,7500,55000,True
6,Chevrolet,Bolt EUV,2022.0,2024,EV,7500,55000,True
7,Chevrolet,Silverado EV,NaN,2024,EV,7500,80000,True
8,Chrysler,Pacifica PHEV,2022.0,2023,PHEV,7500,80000,True
9,Ford,E-Transit,2022.0,2023,EV,7500,80000,True


In [11]:
tax_rebate_per_model_df.columns

Index(['manufacturer', 'model', 'model_year_introduced', 'curent_model-year',
       'vehicle_type', 'tax_credit_amount', 'msrp_limit', 'assembled_in _usa'],
      dtype='object')

In [12]:
selected_columns_for_tax_rebate_model= ['manufacturer', 'model', 'curent_model-year',
       'vehicle_type', 'tax_credit_amount', 'msrp_limit', 'assembled_in _usa']
tax_rebate_model_compressed_df = tax_rebate_per_model_df[selected_columns_for_tax_rebate_model]
tax_rebate_model_compressed_df

,manufacturer,model,curent_model-year,vehicle_type,tax_credit_amount,msrp_limit,assembled_in _usa
0,Audi,Q5 TFSI e Quattro (PHEV),2023,PHEV,7500,80000,True
1,BMW,330e,2023,PHEV,5836,55000,True
2,BMW,X5 xDrive45e,2023,PHEV,7500,80000,True
3,BMW,X5 xDrive50e,2024,PHEV,7500,80000,True
4,Cadillac,Lyriq,2024,EV,7500,80000,True
5,Chevrolet,Bolt,2024,EV,7500,55000,True
6,Chevrolet,Bolt EUV,2024,EV,7500,55000,True
7,Chevrolet,Silverado EV,2024,EV,7500,80000,True
8,Chrysler,Pacifica PHEV,2023,PHEV,7500,80000,True
9,Ford,E-Transit,2023,EV,7500,80000,True


In [13]:
filtered_df = tax_rebate_model_compressed_df[tax_rebate_model_compressed_df['vehicle_type'] == 'EV']
filtered_df

,manufacturer,model,curent_model-year,vehicle_type,tax_credit_amount,msrp_limit,assembled_in _usa
4,Cadillac,Lyriq,2024,EV,7500,80000,True
5,Chevrolet,Bolt,2024,EV,7500,55000,True
6,Chevrolet,Bolt EUV,2024,EV,7500,55000,True
7,Chevrolet,Silverado EV,2024,EV,7500,80000,True
9,Ford,E-Transit,2023,EV,7500,80000,True
12,Ford,F-150 Lightning,2023,EV,7500,80000,True
13,Ford,Mustang Mach-E,2023,EV,7500,80000,True
14,Genesis,Electrified GV70,2024,EV,7500,80000,True
20,Mercedes-Benz,EQE 350 SUV,2023,EV,7500,80000,True
21,Nissan,Leaf S,2023,EV,7500,55000,True


In [14]:
import requests

city = input("Where are you located? (Please enter following format: City, State Abbreviation, USA) Example: Edison, NJ, USA")
limit =input("How many charging station?")

url = "https://ev-charge-finder.p.rapidapi.com/search-by-location"

querystring = {"near":city, "limit":limit}

headers = {
	"X-RapidAPI-Key": RapidAPIKey,
	"X-RapidAPI-Host": "ev-charge-finder.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring).json()

#print(response.json())
# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "data": [
        {
            "address_components": {
                "city": "Jersey City",
                "country": "US",
                "district": "The Waterfront",
                "state": "New Jersey",
                "street_address": "30 Mall Dr W",
                "zipcode": "07310"
            },
            "connectors": [
                {
                    "available": 0,
                    "kw": 50,
                    "speed": "Fast",
                    "total": 2,
                    "type": "CHAdeMO"
                },
                {
                    "available": 0,
                    "kw": 50,
                    "speed": "Fast",
                    "total": 2,
                    "type": "CCS"
                },
                {
                    "available": 0,
                    "kw": 3.8,
                    "speed": "Slow",
                    "total": 1,
                    "type": "J1772"
                }
            ],
            "f

In [15]:
data_point = response['data']
data_point
len(data_point)

#data_point[0]['name']
data_point[1]['connectors'][0]['speed']

'Slow'

In [27]:
data_point = response['data']

charging_data = []

for i in range(len(data_point)):

    name = (data_point[i]['name'])
    address = (data_point[i]['formatted_address'])
    latitude = (data_point[i]['latitude'])
    longitude = (data_point[i]['longitude'])
    try:
        charging_speed = (data_point[i]['connectors'][0]['speed'])
    except:
        charging_speed = "Not Applicable"

    charging_data.append({"name": name, 
                          "Address": address, 
                          "Lat": latitude, 
                          "Lng": longitude ,
                          "Charging Speed": charging_speed})
    
charging_data_df = pd.DataFrame(charging_data)
charging_data_df

charging_data_df.to_csv('charging_data.csv', index=False)
    

In [20]:

name = []
address = []
latitude = []
longitude = []
charging_speed = [] 


for i in range(len(data_point)):

    print(data_point[i]['name'])
    name.append(data_point[i]['name'])
    address.append(data_point[i]['formatted_address'])
    latitude.append(data_point[i]['latitude'])
    longitude.append(data_point[i]['longitude'])
    charging_speed.append(data_point[i]['connectors'][0]['speed'])
    print(data_point[i]['connectors'][0]['speed'])


EVgo Charging Station
Fast
ChargePoint Charging Station
Slow
Greenspot Charging Station
Fast
Tesla Supercharger
Fast
Electric Vehicle Charging Station


IndexError: list index out of range

In [ ]:
name

In [ ]:
#Convert JSON to DataFrame Using read_json()
df = pd.DataFrame(response)
print(df)

## Step 2: Perform data preparation & cleaning

## Step 3: Data Analysis

## Step 4: Data Visualization

## Ask & answer questions

## Conclusion